## 02. 기계학습(머신러닝) 기초 - 실습

### 1) 기본 Package 설정

앞으로 공부하면서 사용할 Package들을 절차에 맞춰 미리 소개하겠다.  
필요에 따라 각각의 속도에 맞춰 pacakge를 새롭게 설치해도 되지만,   
그냥 공부가 목적이라면 하나의 file로 만들어 가장 앞쪽에 미리 적어 놓는 것이 더욱 효율 적일 것이다.  

참고로 나는 이미 사용할 모델과 data가 있어서 모든 package가 필요하지는 않기에 나에게 필요하지 않은 package들과 추 후에 사용할 모델에 대해서 주석처리해두었다.

In [ ]:
# 기본
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # 그래프

# # 데이터 전처리
# from sklearn import datasets    # sklearn에 있는 data import
# from sklearn.model_selection import train_test_split    # 훈련과 테스트를 위한 데이터 분리
# from sklearn.preprocessing import StandardScaler    # 연속 변수 표준화
# from sklearn.preprocessing import LabelEncoder      # 범주형 변수 수치화

# # 분류 모델
# from sklearn.naive_bayes import GaussianNB      # 나이브 베이즈 분류
# from sklearn.tree import DecisionTreeClassifier     # 의사결정나무
# from sklearn.ensemble import RandomForestClassifier     # 랜덤 포레스트
# from sklearn.linear_model import LogisticRegression     # 로지스틱 회귀분석
# from sklearn.svm import SVC      # SVM(서포트벡터머신)
# from sklearn.neural_network import MLPClassifier    # 다층 인공신경망

# 모델 검정
from sklearn.metrics import confusion_matrix, classification_report # 모델 검정
from sklearn.metrics import accuracy_score, precision_score, f1_score, make_scorer

### 2) 데이터 가져오기

### 3) 데이터 전처리

### 4) 훈련/검증용 데이터 분할

### 5) 모델 구축

### 6) 모델 검정

### 7) 최적화